# The temperature quantiles

In [1]:
from pathlib import Path

import numpy as np
import xarray as xr

import dask
from dask.diagnostics import ProgressBar

sys.path.insert(0, '../..')
from util import DATA_DIR

In [4]:
MAX_YEAR = 2020
REFERENCE_YEAR_START = 1986
REFERENCE_YEAR_END = 2005

TEMPERATURES_FOLDER = DATA_DIR / "era5_0.25deg/daily_temperature_summary"


In [5]:
def year_from_filename(name):
    return int(name.split('_')[-3][-4:])


In [6]:
CLIMATOLOGY_QUANTILES_FOLDER = DATA_DIR / "era5_0.25deg/quantiles"

In [5]:
# CLIMATOLOGY_QUANTILES_FOLDER.mkdir()

In [7]:
# QUANTILES = np.arange(0, 1, 0.01)
QUANTILES = [0.05,0.95]

## Tmax

In [7]:
t_var = 'tmax'

In [8]:
file_list = sorted([p for p in TEMPERATURES_FOLDER.rglob('*.nc')
                    if year_from_filename(p.name) >= REFERENCE_YEAR_START 
                    and year_from_filename(p.name) <= REFERENCE_YEAR_END 
                   ]
                  )

In [9]:
# with dask.config.set(**{'array.slicing.split_large_chunks': False}):
daily_temperatures = xr.open_mfdataset(file_list, combine='by_coords', chunks ={'latitude': 100, 'longitude': 100})['t_max']

daily_temperatures = daily_temperatures.chunk({'time': -1})

In [10]:
daily_temperatures

<xarray.DataArray 't_max' (time: 7281, latitude: 721, longitude: 1440)>
dask.array<rechunk-merge, shape=(7281, 721, 1440), dtype=float32, chunksize=(7281, 100, 100), chunktype=numpy.ndarray>
Coordinates:
  * longitude  (longitude) float64 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float64 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * time       (time) datetime64[ns] 1986-01-01 1986-01-02 ... 2005-12-31
    number     int64 0
    surface    float64 0.0
    step       timedelta64[ns] 00:00:00

In [11]:
CLIMATOLOGY_QUANTILES = (CLIMATOLOGY_QUANTILES_FOLDER / 
                         f'daily_{t_var}_quantiles_{"_".join([str(int(100*q)) for q in QUANTILES])}_1986-2005.nc')

In [12]:
daily_quantiles = daily_temperatures.quantile(QUANTILES, dim='time')

In [13]:
with dask.config.set(scheduler='processes'), ProgressBar():
    daily_quantiles = daily_quantiles.compute()
    daily_quantiles.to_netcdf(CLIMATOLOGY_QUANTILES)

[########################################] | 100% Completed | 50min  6.9s


In [14]:
with ProgressBar():
    daily_quantiles.to_netcdf(CLIMATOLOGY_QUANTILES)

## Tmin

In [15]:
t_var = 'tmin'

In [16]:
file_list = sorted([p for p in TEMPERATURES_FOLDER.rglob('*.nc')
                    if year_from_filename(p.name) >= REFERENCE_YEAR_START 
                    and year_from_filename(p.name) <= REFERENCE_YEAR_END 
                   ]
                  )

In [17]:
# with dask.config.set(**{'array.slicing.split_large_chunks': False}):
daily_temperatures = xr.open_mfdataset(file_list, combine='by_coords', chunks ={'latitude': 100, 'longitude': 100})['t_min']
daily_temperatures = daily_temperatures.chunk({'time': -1})

In [18]:
CLIMATOLOGY_QUANTILES = (CLIMATOLOGY_QUANTILES_FOLDER / 
                         f'daily_{t_var}_quantiles_{"_".join([str(int(100*q)) for q in QUANTILES])}_1986-2005.nc')

In [19]:
daily_quantiles = daily_temperatures.quantile(QUANTILES, dim='time')

with dask.config.set(scheduler='processes'), ProgressBar():
    daily_quantiles = daily_quantiles.compute()
    daily_quantiles.to_netcdf(CLIMATOLOGY_QUANTILES)

[########################################] | 100% Completed | 51min  2.8s
